In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import math
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
# import addcopyfighandler

import seaborn as sns

from IPython.core.display import HTML

sys.path.append("../RD-systems-and-test-benches/utils")
import data_processing as dp

import model_fsolve as modf

import scipy.optimize as sco
import scipy.interpolate as sci

import simu as sim

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl

import torch
import torch_model as modto

In [ ]:
path = r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs"
file_name = r"\Garcia_Guendulain.xlsx"

In [ ]:
hx, par, cond = fe.initialize(path, file_name)

In [ ]:
df, res, df_PL, testings = modf.PL_fsolve(par, cond, series=[350/3600000, 50/3600000, 0.1])

In [ ]:
rho = cond["rho"]
eta = cond["eta"]
ep = par["roughness"]
Din = par["D_man"]
L_man = par["L_man"]
fin = fds.friction.friction_factor(Re = fds.core.Reynolds(df_u.loc[0]['uin'],Din,rho,mu=eta),eD = ep/Din)
(rho/2)*fin*L_man*(df_u.loc[8]['uin'])**2/Din

In [ ]:
500*(df_u.loc[8]['uin']**2 -df_u.loc[0]['uin']**2)

In [ ]:
list_QF = np.array([50,200,350,500,650])/3600000
list_QF_out = np.array([50,200,350])/3600000
list_alpha = [0.1, 0.4, 0.7, 1]
df = fe.testing_series(r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Garcia_testings.xlsx", list_QF, list_QF_out, list_alpha, par, cond)

In [ ]:
df2 = pd.read_excel('G:\Drive partagés\BU04-Innovation\PVT-PL-model\Garcia_testings.xlsx')

In [ ]:
df2[df2['QF_out']==list_QF_out[0]]

In [ ]:
dplt.K_abaque(df2, 'Kyout')


In [ ]:
TUV = pd.read_excel('G:\Drive partagés\BU04-Innovation\Hydraulique\PL_database\\230727_TUV_V4.41_x1_Z_water.xlsx')

In [ ]:
# read excel file
excel_file = pd.ExcelFile(r'G:\Drive partagés\BU04-Innovation\Hydraulique\components\230725_PL_V4.41.xlsx')

# create dictionary with dataframes
df_dict = {}
for sheet_name in excel_file.sheet_names:
    df_dict[sheet_name] = excel_file.parse(sheet_name)
    df_dict[sheet_name].drop(columns='Unnamed: 0',inplace=True)

In [ ]:
for i in range(4):
    key = list(df_dict.keys())[i]
    list_Vdot = np.array(df_dict[key]['Vdot_per_panel'])
    list_Dv = list_Vdot*(i+1)/3600000

    path = r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs"
    file_name = f"\V4.41_x{i+1}.xlsx"
    hx, par, cond = fe.initialize(path, file_name)
    list_PL, list_tabl = modf.PL_fsolve_range(par, cond, list_Dv[1::39])

    plt.plot(list_Vdot[1::39], list_PL/1000, label='Simulation')
    plt.plot(list_Vdot, np.array(df_dict[key]['PL']), label='Experiments')
    plt.xlabel('Vdot par panneau [L/h]')
    plt.ylabel('PL [kPa]')
    plt.legend()
    plt.show()



In [ ]:
list(df_dict.keys())

In [ ]:
for k in df_dict.keys():
    print(df_dict[k]['PL'])